In [ ]:
import io
import os
import re
import zipfile
from argparse import ArgumentParser
from pathlib import Path
from typing import Final

import cv2
import numpy as np
import numpy.typing as npt
import requests
import pycolmap

from tqdm import tqdm

import rerun as rr

rr.init("RECONSTRUCTION")


folder_path = '/home/sergio/programming/misc/image-matching-challenge-2024/Hierarchical-Localization/'

# smf_path = Path(folder_path, 'datasets/image-matching-challenge-2024/train/dioscuri/sfm').resolve()
# imgs_path = Path(folder_path, 'datasets/image-matching-challenge-2024/train/dioscuri/images').resolve()

smf_path = Path(folder_path, '../temp/aliked2k+disk+sift-rot-pixsfm-sci-loc/dioscuri/dioscuri/sparse/').resolve()
imgs_path = Path(folder_path, '../temp/aliked2k+disk+sift-rot-pixsfm-sci-loc/dioscuri/dioscuri/images/').resolve()

# smf_path = Path(folder_path, '../temp/aliked2k+disk+sift-rot-pixsfm-sci-loc/church/church/sparse/').resolve()
# imgs_path = Path(folder_path, '../temp/aliked2k+disk+sift-rot-pixsfm-sci-loc/church/church/images/').resolve()

In [ ]:
DESCRIPTION = """
# Sparse Reconstruction by COLMAP
This example was generated from the output of a sparse reconstruction done with COLMAP.

[COLMAP](https://colmap.github.io/index.html) is a general-purpose Structure-from-Motion (SfM) and Multi-View Stereo
(MVS) pipeline with a graphical and command-line interface.

In this example a short video clip has been processed offline by the COLMAP pipeline, and we use Rerun to visualize the
individual camera frames, estimated camera poses, and resulting point clouds over time.

## How it was made
The full source code for this example is available
[on GitHub](https://github.com/rerun-io/rerun/blob/latest/examples/python/structure_from_motion/main.py).

### Images
The images are logged through the [rr.Image archetype](https://www.rerun.io/docs/reference/types/archetypes/image)
to the [camera/image entity](recording://camera/image).

### Cameras
The images stem from pinhole cameras located in the 3D world. To visualize the images in 3D, the pinhole projection has
to be logged and the camera pose (this is often referred to as the intrinsics and extrinsics of the camera,
respectively).

The [rr.Pinhole archetype](https://www.rerun.io/docs/reference/types/archetypes/pinhole) is logged to
the [camera/image entity](recording://camera/image) and defines the intrinsics of the camera. This defines how to go
from the 3D camera frame to the 2D image plane. The extrinsics are logged as an
[rr.Transform3D archetype](https://www.rerun.io/docs/reference/types/archetypes/transform3d) to the
[camera entity](recording://camera).

### Reprojection error
For each image a [rr.Scalar archetype](https://www.rerun.io/docs/reference/types/archetypes/scalar)
containing the average reprojection error of the keypoints is logged to the
[plot/avg_reproj_err entity](recording://plot/avg_reproj_err).

### 2D points
The 2D image points that are used to triangulate the 3D points are visualized by logging
[rr.Points3D archetype](https://www.rerun.io/docs/reference/types/archetypes/points2d)
to the [camera/image/keypoints entity](recording://camera/image/keypoints). Note that these keypoints are a child of the
[camera/image entity](recording://camera/image), since the points should show in the image plane.

### Colored 3D points
The colored 3D points were added to the scene by logging the
[rr.Points3D archetype](https://www.rerun.io/docs/reference/types/archetypes/points3d)
to the [points entity](recording://points):
```python
rr.log("points", rr.Points3D(points, colors=point_colors), rr.AnyValues(error=point_errors))
```
**Note:** we added some [custom per-point errors](recording://points) that you can see when you
hover over the points in the 3D view.
""".strip()


In [ ]:
FILTER_MIN_VISIBLE = 60
def scale_camera(camera, resize: tuple[int, int]) -> tuple[pycolmap.Camera, npt.NDArray[np.float_]]:
    """Scale the camera intrinsics to match the resized image."""
    assert camera.model == "PINHOLE"
    new_width = resize[0]
    new_height = resize[1]
    scale_factor = np.array([new_width / camera.width, new_height / camera.height])

    # For PINHOLE camera model, params are: [focal_length_x, focal_length_y, principal_point_x, principal_point_y]
    new_params = np.append(camera.params[:2] * scale_factor, camera.params[2:] * scale_factor)

    return (pycolmap.Camera(camera.id, camera.model, new_width, new_height, new_params), scale_factor)



def read_and_log_sparse_reconstruction(rec_path: Path, img_path: Path, filter_output: bool, resize: tuple[int, int] | None) -> None:
    print("Reading sparse COLMAP reconstruction")
    reconstruction = pycolmap.Reconstruction(rec_path)
    cameras = reconstruction.cameras
    images = reconstruction.images
    points3D = reconstruction.points3D
    print("Building visualization by logging to Rerun")

    if filter_output:
        # Filter out noisy points
        points3D = {id: point for id, point in points3D.items() if point.color.any() and len(point.image_ids) > 2}

    # rr.log("description", rr.TextDocument(DESCRIPTION, media_type=rr.MediaType.MARKDOWN), timeless=True)
    rr.log("/", rr.ViewCoordinates.RIGHT_HAND_Y_DOWN, timeless=True)
    rr.log("plot/avg_reproj_err", rr.SeriesLine(color=[240, 45, 58]), timeless=True)
    
    # Iterate through images (video frames) logging data related to each frame.
    ii=0
    
    list_visible_xyzs = []
    all_point = []
    all_point_colors = []
    all_point_errors = []
    for image in tqdm(sorted(images.values(), key=lambda im: im.name)):  # type: ignore[no-any-return]
        image_file = img_path / image.name.replace('.jpg', '.png')
        if not os.path.exists(image_file):
            continue
        # print (image_file)

        # COLMAP sets image ids that don't match the original video frame
        idx_match = re.search(r"\d+", image.name)
        assert idx_match is not None
        frame_idx = int(idx_match.group(0))


        camera = cameras[image.camera_id]
        if resize:
            camera, scale_factor = scale_camera(camera, resize)
        else:
            scale_factor = np.array([1.0, 1.0])

        visible_ids = [id_ for id_ in points3D.keys() if image.has_point3D(id_) ]
        

        if filter_output and len(visible_ids) < FILTER_MIN_VISIBLE:
            continue

        visible_xyzs = [points3D[idx] for idx in visible_ids]
        visible_xys = np.array([x.xy for x in image.get_valid_points2D()])
        
        list_visible_xyzs.extend(visible_xyzs)
        
        if resize:
            visible_xys *= scale_factor

        rr.set_time_sequence("frame", frame_idx)
        try:
            points = [point.xyz for point in visible_xyzs]
        except Exception as e:
            print (e)
            continue
        point_colors = [point.color for point in visible_xyzs]
        point_errors = [point.error for point in visible_xyzs]

        rr.log("plot/avg_reproj_err", rr.Scalar(np.mean(point_errors)))

        rr.log("points", rr.Points3D(points, colors=point_colors), rr.AnyValues(error=point_errors))    
        
        all_point.extend(points)
        all_point_colors.extend(point_colors)
        all_point_errors.extend(point_errors)    
        
        rr.log("points", rr.Points3D(all_point, colors=all_point_colors), rr.AnyValues(error=all_point_errors))    
        
        # COLMAP's camera transform is "camera from world"
        try:
            rr.log(
                "camera", rr.Transform3D(translation=image.cam_from_world.translation,
                                        rotation=rr.Quaternion(xyzw=image.cam_from_world.rotation.quat), from_parent=True)
            )
        except:
            print(f"Image {frame_idx} not valid")
            
        rr.log("camera", rr.ViewCoordinates.RFD, timeless=True)  # X=Right, Y=Down, Z=Forward
       

        
        try:
            # Log camera intrinsics
            # assert str(camera.model) in  ["CameraModelId.SIMPLE_PINHOLE", "CameraModelId.PINHOLE"]
            if str(camera.model) == "CameraModelId.SIMPLE_PINHOLE":
                print("camera pin hole")
                rr.log(
                    "camera/image",
                    rr.Pinhole(
                        resolution=[camera.width, camera.height],
                        focal_length=[camera.params[0], camera.params[0]],
                        principal_point=camera.params[1:],
                    ),
                )
            else:
                print("other camera")
                rr.log(
                    "camera/image",
                    rr.Pinhole(
                        resolution=[camera.width, camera.height],
                        focal_length=camera.params[:2],
                        principal_point=camera.params[2:],
                    ),
                )
                
        except:
            rr.log(
                    "camera/image",
                    rr.Pinhole(
                        resolution=[camera.width, camera.height],
                        focal_length=[camera.params[0], camera.params[0]],
                    ),
            )
        
        if resize:
            bgr = cv2.imread(str(image_file))
            bgr = cv2.resize(bgr, resize)
            rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
            rr.log("camera/image", rr.Image(rgb).compress(jpeg_quality=75))
        else:
            rr.log("camera/image", rr.ImageEncoded(path=img_path / image.name.replace('.jpg', '.png')))
        rr.log("camera/image/keypoints", rr.Points2D(visible_xys, colors=[34, 138, 167]))
          
                     
    print ("Now preparing visualization engine")

In [ ]:
# Whatever you want to visualize in notebook, you should start the rec = rr.memory_recording()
rec = rr.memory_recording()
# rec = rr.notebook_show()
read_and_log_sparse_reconstruction(Path(smf_path), Path(imgs_path), filter_output=False, resize=None)
# And after it finishes - show it by just calling it  rec
rr.notebook_show()
# rr.start_web_viewer_server()
